<a href="https://colab.research.google.com/github/InTEGr8or/jupyter-fun/blob/master/nCov19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Infection Rates per day

The percent rates in the sheet at the bottom are approximations. Data seems to be released about twice a day. Sometimes there is a lag, and they release it at different times.

Optimally, the time of the update would be taken into account and prorated per hour and multiplied by the number of hours difference but I am just starting to learn Python so there are probably a lot of improvements that could be made.

Data is collected by Johns Hopkins in Baltimore and published here: [nCov19 contagion](https://gisanddata.maps.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6)



## This first section sets up the imports and some parsing functions.


In [25]:
try:
  import pygsheets
except:
  !pip install pygsheets
  import pygsheets
try:
  import numpy
except:
  !pip install numpy
  import numpy
from dateutil import parser
try:
  import requests
except:
  !pip install requests
  import requests

try:
  from bs4 import BeautifulSoup
except:
  !pip install beautifulsoup4
  from bs4 import BeautifulSoup
  
import gspread
from oauth2client.client import GoogleCredentials
import pandas as pd

from google.colab import auth
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

source_url = "https://docs.google.com/spreadsheets/d/1yZv9w9zRKwrGTaR-YzmAqMefw4wMlaXocejdxZaTs6w"
test = "curl https://docs.google.com/spreadsheets/d/1yZv9w9zRKwrGTaR-YzmAqMefw4wMlaXocejdxZaTs6w/htmlview"

pd.set_option('display.max_rows', 100)
def get_sheets(sheet_url):
  page = requests.get(sheet_url)
  worksheets = BeautifulSoup(page.text)
  tpath = "//div[@id='sheets-viewport']/div //table"
  # the requests just gets the ajax scripts at this point.
  print(page.text.find('sheets-veiwport'))
  # thats all the sheets
  pass
get_sheets(source_url + '/htmlview')

def sdate(sheet_rows):
  header = sheet_rows[0]
  update_col = [i for i, item in enumerate(header) if "Update" in item or "update" in item]
  return parser.parse(sheet_rows[3][update_col[0]])

def rdate(sheet_rows):
  tdate = sdate(sheet_rows)
  return f'{tdate.strftime("%B")[:3]}{tdate.day}'

def rdtime(sheet_rows):
  tdate = parser.parse(sheet_rows[3][sheet_rows[0].index('Last Update')])

def hotten(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    if '%' in s:
      print(s)
      is_max = float(s.replace('%', '')) > 20
      return ['background-color: yellow' if v else '' for v in is_max]
    else:
      return 'background-color: black'

-1


## Get The Sheets and Loop Through

In [0]:
#TODO: Open the public sheet in a way that doesn't require authentication, like the pd.read_csv() below.  
worksheets = gc.open_by_url(source_url).worksheets()
prows = worksheets[0].get_all_values()
pdate = None
# df = pd.read_csv(source_url, dtype={0:'str', 1:'str', 2:'str'})
df = pd.DataFrame.from_records(prows[1:], columns=prows[0])
df.rename(columns={"Confirmed": rdate(prows)}, inplace=True)
for worksheet in worksheets:
  crows = worksheet.get_all_values()
  if sdate(crows).day != sdate(prows).day:
    try:
      prov_col = prows[0].index('Province/State')
      cnt_col = [i for i,item in enumerate(prows[0]) if "Country" in item][0]
      conf_col = prows[0].index('Confirmed')
      dt_col = rdate(crows)
      prior_dt_col = rdate(prows)
      df[f'{prior_dt_col}%'] = '0%'
      df[dt_col] = 0
      for i, df_row in df.iterrows():
        # TODO: Find the row for the df province
        for j, row in enumerate(prows):
          if row[cnt_col] == df_row['Country/Region'] and row[prov_col] == df_row['Province/State']:
            if row[conf_col].isnumeric():
              cconf = int(row[conf_col])
              pconf = int(df_row[prior_dt_col])
              df.at[i, dt_col] = row[conf_col]
              # if "Hubei" in row[prov_col]:
              #   print(f'{i}:{prior_dt_col}% = {pconf}/{cconf} = {pconf / cconf}')
              df.at[i, f'{prior_dt_col}%'] = f'{(round(pconf / cconf * 100) - 100)}%'
    except:
      print(prows[:2])
  prows = crows




## Print Out the Data


In [27]:
df.drop(labels='Last Update',axis=1)#.style.applymap(hotten)

,Province/State,Country/Region,Feb2,Deaths,Recovered,Feb2%,Feb1,Feb1%,Jan31,Jan31%,Jan30,Jan30%,Jan29,Jan29%,Jan28,Jan28%,Jan27,Jan27%,Jan26,Jan26%,Jan25,Jan25%,Jan24,Jan24%,Jan23,Jan23%,Jan22,Jan22%,Jan21
0,Hubei,Mainland China,9074,294,215,0%,9074,27%,7153,23%,5806,18%,4903,38%,3554,31%,2714,91%,1423,34%,1058,45%,729,33%,549,24%,444,0%,0
1,Zhejiang,Mainland China,661,0,23,0%,661,10%,599,11%,538,26%,428,45%,296,71%,173,35%,128,23%,104,68%,62,44%,43,59%,27,0%,0
2,Guangdong,Mainland China,604,0,14,0%,604,13%,535,23%,436,23%,354,28%,277,34%,207,37%,151,36%,111,42%,78,47%,53,66%,32,0%,0
3,Henan,Mainland China,493,2,4,0%,493,17%,422,20%,352,27%,278,35%,206,23%,168,31%,128,54%,83,159%,32,256%,9,80%,5,0%,0
4,Hunan,Mainland China,463,0,8,0%,463,19%,389,17%,332,20%,277,25%,221,55%,143,43%,100,45%,69,60%,43,79%,24,167%,9,0%,0
5,Anhui,Mainland China,340,0,5,0%,340,14%,297,25%,237,18%,200,32%,152,43%,106,51%,70,17%,60,54%,39,160%,15,67%,9,0%,0
6,Jiangxi,Mainland China,333,0,10,0%,333,16%,286,19%,240,48%,162,49%,109,51%,72,50%,48,33%,36,100%,18,157%,7,0%,7,0%,0
7,Chongqing,Mainland China,262,1,3,0%,262,6%,247,17%,211,16%,182,24%,147,11%,132,20%,110,47%,75,32%,57,111%,27,200%,9,0%,0
8,Jiangsu,Mainland China,236,0,6,0%,236,17%,202,20%,168,30%,129,30%,99,41%,70,49%,47,42%,33,83%,18,100%,9,80%,5,0%,0
9,Sichuan,Mainland China,231,1,3,0%,231,12%,207,17%,177,25%,142,31%,108,20%,90,30%,69,57%,44,57%,28,87%,15,88%,8,0%,0
